# rough work

# extraction of the text from the docuemnt 

In [12]:
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

In [ ]:
document_name = ['Adani  Mundra Port agreement for operations',
                 'Bahuli_IOCL_compressed',
                 'Banthra_Kribhco_compressed',
                 'CONCOR Agreement for operations_compressed',
                 'GCT Pathri_Concor_compressed',
                 'Kundanganj_Reliance_compressed',
                 'MMLP Barhi_Concor_compressed (1)']



In [1]:
import boto3
import time


def pdf_to_text(name):
    textract = boto3.client("textract", region_name="ap-south-1")

    response = textract.start_document_text_detection(
        DocumentLocation={
            "S3Object": {
                "Bucket": "textract-input-happy",
                "Name": f'{name}.pdf'
            }
        }
    )

    job_id = response["JobId"]
    print("Job started:", job_id)

    while True:
        status = textract.get_document_text_detection(JobId=job_id)
        job_status = status["JobStatus"]

        print("Status:", job_status)

        if job_status in ["SUCCEEDED", "FAILED"]:
            break

        time.sleep(5)

    blocks = []
    next_token = None

    while True:
        if next_token:
            response = textract.get_document_text_detection(
                JobId=job_id,
                NextToken=next_token
            )
        else:
            response = textract.get_document_text_detection(JobId=job_id)

        blocks.extend(response["Blocks"])

        next_token = response.get("NextToken")
        if not next_token:
            break

    lines = [
        block["Text"]
        for block in blocks
        if block["BlockType"] == "LINE"
    ]

    full_text = "\n".join(lines)

    with open(f'extracted_text/{name}.txt', "w", encoding="utf-8") as f:
        f.write(full_text)

In [4]:
for name in document_name:
    pdf_to_text(name)

Job started: 2fb71aa3c82292b731d66a72d9f733bdbd0a9c5961c5f2a31fc0fc8feae3a2ea
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: SUCCEEDED
Job started: ab50c18b9020d9f352d1804a4c5a855a554231f88cd13e35428b8332ed6fbe0c
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: SUCCEEDED
Job started: 3cb9611973b8648f080a4327caddd088f10dc70cde5d60f2fe27071951e4fe50
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: SUCCEEDED
Job started: 5e3d0e97aad1d6f291548540ae155b71015692e733a63dbe49241a87b037d7e9
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: IN_PROGRESS
Status: SUCCEEDED
Job started: f9b4a0a7fa1f7553dbb7b10931136847ebebba2c30cf0d5d0d531b7520ff555b
Status: IN_PROGRES

# creating vectorDB of all fil

In [8]:
from pathlib import Path
from utils.vecDb import LegalVectorDB

TEXT_DIR = Path("extracted_text")
VECTOR_DB_DIR = Path("vectorDB")
VECTOR_DB_DIR.mkdir(exist_ok=True)

DOCUMENT_NAMES = [
    "Adani  Mundra Port agreement for operations",
    "Bahuli_IOCL_compressed",
    "Banthra_Kribhco_compressed",
    "CONCOR Agreement for operations_compressed",
    "GCT Pathri_Concor_compressed",
    "Kundanganj_Reliance_compressed",
    "MMLP Barhi_Concor_compressed (1)"
]


def build_all_vector_dbs():
    """
    One-time ingestion step.
    Creates one vector DB per document using document_name as folder name.
    """

    for doc_name in DOCUMENT_NAMES[3:]:
        print(f"📦 Building vector DB for: {doc_name}")

        file_path = TEXT_DIR / f"{doc_name}.txt"
        if not file_path.exists():
            raise FileNotFoundError(f"Missing file: {file_path}")

        with open(file_path, "r", encoding="utf-8") as f:
            text = f.read()

        vecdb = LegalVectorDB()
        vecdb.ingest_document(text, source_name=doc_name)
        vecdb.build_index()

        db_path = VECTOR_DB_DIR / doc_name
        vecdb.save(str(db_path))

        print(f"✅ Saved vector DB at: {db_path}")


In [9]:
build_all_vector_dbs()

📦 Building vector DB for: CONCOR Agreement for operations_compressed
✅ Saved vector DB at: vectorDB/CONCOR Agreement for operations_compressed
📦 Building vector DB for: GCT Pathri_Concor_compressed
✅ Saved vector DB at: vectorDB/GCT Pathri_Concor_compressed
📦 Building vector DB for: Kundanganj_Reliance_compressed
✅ Saved vector DB at: vectorDB/Kundanganj_Reliance_compressed
📦 Building vector DB for: MMLP Barhi_Concor_compressed (1)
✅ Saved vector DB at: vectorDB/MMLP Barhi_Concor_compressed (1)


In [13]:
import os
from pathlib import Path
from typing import Dict
import json
from pathlib import Path
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import re

from utils.vecDb import LegalVectorDB   # your vecDB.py


# =====================================================
# CONFIG
# =====================================================

DOCUMENT_NAMES = [
    "Adani  Mundra Port agreement for operations",
    "Bahuli_IOCL_compressed",
    "Banthra_Kribhco_compressed",
    "CONCOR Agreement for operations_compressed",
    "GCT Pathri_Concor_compressed",
    "Kundanganj_Reliance_compressed",
    "MMLP Barhi_Concor_compressed (1)"
]

TEXT_DIR = Path("extracted_text")
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)


# =====================================================
# 12 SECTION DEFINITIONS
# =====================================================

SECTIONS: Dict[int, Dict[str, str]] = {
    1: {"name": "Context & Objective",
        "query": "Extract the background, policy rationale, and purpose of this PPP concession agreement, including the problem it seeks to address, the public interest objective, the scope of the concession, and the intended outcomes for infrastructure service delivery and market efficiency"},
    2: {"name": "Scope of Concession & Rights Granted",
        "query": "Extract clauses defining the scope of the concession, activities permitted to the concessionaire, rights granted to develop, operate, maintain, manage, or commercially exploit the project assets, and any explicit limitations or exclusions."},
    3: {"name": "Asset Ownership & Control",
        "query": "Extract clauses specifying ownership of project assets, land, and equipment, rights of use versus title, and provisions allocating operational control, access, supervision, and decision-making authority."},
    4: {"name": "Regulatory & Operational Compliance",
        "query": "Extract clauses imposing regulatory, operational, and compliance obligations, including approvals, directions, inspections, reporting requirements, standards, and the extent of supervisory or micromanagement authority over operations."},
    5: {"name": "Concession Period & Extension",
        "query": "Extract clauses defining the concession term, commencement and expiry, conditions for extension or renewal, discretion of the authority, and any limits or uncertainty affecting the investment horizon."},
    6: {"name": "Tariff & Revenue Flexibility",
        "query": "Extract all provisions governing how prices, tariffs, fees, or user charges are determined, revised, approved, or controlled; the extent of pricing discretion or regulation; revenue-sharing or levy mechanisms; indexation or adjustment formulas; and any restrictions affecting commercial revenue generation or innovation."},
    7: {"name": "Demand & Traffic Risk Allocation",
        "query": "Extract clauses allocating demand, volume, traffic, or throughput risk, including minimum or assured traffic, exclusivity or non-exclusivity, diversion rights, competing facilities, capacity commitments, and any guarantees or disclaimers affecting traffic levels."},
    8: {"name": "Change in Law & Policy Risk",
        "query": "Extract clauses addressing change in law, policy, regulation, or interpretation; allocation of resulting costs or benefits; compensation, relief, or adjustment mechanisms; and protections against adverse governmental or regulatory actions affecting the project."},
    9: {"name": "Relief Structure",
        "query": "Extract clauses providing relief from contractual obligations due to force majeure, change in circumstances, or external shocks, including suspension, extension of time, cost or tariff adjustment, termination relief, and conditions or limitations on such relief."},
    10: {"name": "Termination & Step-in Rights",
         "query": "Extract clauses governing termination events, defaults and cure periods, termination consequences, compensation or payout mechanisms, lender step-in or substitution rights, and protections preserving continuity of the project upon default or early termination."},
    11: {"name": "Dispute Resolution & Governing Law",
         "query": "Extract clauses specifying dispute resolution mechanisms, escalation steps, arbitration or court processes, seat and venue, governing law, jurisdiction, timelines, enforceability of awards, and any provisions affecting neutrality or speed of dispute resolution."},
    12: {"name": "Assignment & Financing Flexibility",
         "query": "Extract clauses governing assignment or transfer of rights and obligations, change of control, creation of security interests, financing or refinancing rights, lender protections, substitution or novation, and any approvals or restrictions affecting exit or refinancing."}
}


# =====================================================
# SECTION EXTRACTION PROMPT
# =====================================================

SECTION_PROMPT = ChatPromptTemplate.from_template("""
ROLE:
You are a senior infrastructure PPP legal–economic analyst advising
Government of India policy reform committees and multilateral lenders.

TASK:
Analyze the concession agreement ONLY for the section:
{section_name}

EVIDENCE DISCIPLINE (CRITICAL):
- Every legal conclusion MUST be backed by contract language
- Clearly distinguish between:
  (a) Explicit clauses
  (b) Implicit structure inferred from clauses
  (c) Absence or silence
- Do NOT speculate beyond the document
- If the document is silent, explicitly say so

OUTPUT FORMAT (STRICT — FOLLOW EXACTLY):

SECTION: {section_name}

LEGAL POSITION:
- What the contract explicitly allows, restricts, or mandates
- Clearly indicate if coverage is partial or conditional

ECONOMIC IMPLICATION:
- Commercial consequences strictly flowing from the contract text
- No assumptions beyond documented provisions

BANKABILITY IMPACT:
- Lender and investor perspective based on clarity, enforceability,
  and risk allocation in the contract

CLAUSE EVIDENCE:
- Verbatim excerpts from the document
- Include Article / Clause numbers wherever available
- Do NOT paraphrase in this section

EVIDENCE_STRENGTH: <STRONG | MODERATE | WEAK | NONE>
(one line only, uppercase, choose exactly one)

DOCUMENT CONTEXT:
{context}
""")




# =====================================================
# ANALYSIS FUNCTION
# =====================================================

def analyze_document(document_name: str):
    print(f"\n📄 Processing document: {document_name}")

    # -----------------------------
    # Load document text
    # -----------------------------
    file_path = TEXT_DIR / f"{document_name}.txt"
    if not file_path.exists():
        raise FileNotFoundError(f"Missing file: {file_path}")

    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()

    # -----------------------------
    # Build Vector DB (page / chunk logic is inside LegalVectorDB)
    # -----------------------------
    # vecdb = LegalVectorDB()
    VECTOR_DB_DIR = Path("vectorDB") / document_name
    print(VECTOR_DB_DIR)
    # if not VECTOR_DB_DIR.exists():
    #     raise FileNotFoundError(
    #         f"Vector DB not found for document: {document_name}. "
    #         f"Expected at {VECTOR_DB_DIR}"
    #     )

    vecdb = LegalVectorDB.load(VECTOR_DB_DIR)

    # -----------------------------
    # LLM
    # -----------------------------
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        api_key=os.getenv("OPENAI_API_KEY")
    )

    chain = SECTION_PROMPT | llm | StrOutputParser()

    # -----------------------------
    # Output directory (one folder per document)
    # -----------------------------
    section_out_dir = OUTPUT_DIR / document_name
    section_out_dir.mkdir(parents=True, exist_ok=True)

    # -----------------------------
    # Process each section
    # -----------------------------
    for sec_id, sec in SECTIONS.items():
        print(f"   ➤ Section {sec_id}: {sec['name']}")

        # Retrieve candidate chunks/pages
        docs = vecdb.retrieve_for_section(
            section_query=sec["query"],
            top_k=12  # higher recall for legal analysis
        )

        # Build LLM context
        context = "\n\n---\n\n".join(d.page_content for d in docs)

        # Run LLM
        result = chain.invoke({
            "section_name": sec["name"],
            "context": context
        })

        # -----------------------------
        # Confidence heuristic (simple but useful)
        # -----------------------------
        result_lower = result.lower()

        if "evidence strength:\n- none" in result_lower:
            confidence = "LOW"
        elif "evidence strength:\n- weak" in result_lower:
            confidence = "LOW"
        elif "evidence strength:\n- moderate" in result_lower:
            confidence = "MEDIUM"
        elif "evidence strength:\n- strong" in result_lower:
            confidence = "HIGH"
        else:
            confidence = "UNKNOWN"


        confidence = "UNKNOWN"

        match = re.search(r"EVIDENCE_STRENGTH:\s*(STRONG|MODERATE|WEAK|NONE)", result.upper())

        if match:
            strength = match.group(1)
            if strength == "STRONG":
                confidence = "HIGH"
            elif strength == "MODERATE":
                confidence = "MEDIUM"
            elif strength in ["WEAK", "NONE"]:
                confidence = "LOW"


        # -----------------------------
        # Build JSON output
        # -----------------------------
        section_json = {
            "document": document_name,
            "section_id": sec_id,
            "section_name": sec["name"],
            "query_used": sec["query"],
            "analysis": result,
            "confidence": confidence
        }
        print(result)

        # -----------------------------
        # Write JSON file
        # -----------------------------
        file_name = f"{sec_id:02d}_{sec['name'].replace(' ', '_')}.json"
        section_file = section_out_dir / file_name

        with open(section_file, "w", encoding="utf-8") as f:
            json.dump(section_json, f, indent=2, ensure_ascii=False)

        print(f"      ✓ Saved {file_name}")

    print(f"\n✅ Completed analysis for document: {document_name}")

In [20]:
name = DOCUMENT_NAMES[0]
analyze_document(name)


📄 Processing document: Adani  Mundra Port agreement for operations
vectorDB/Adani  Mundra Port agreement for operations
   ➤ Section 1: Context & Objective
**SECTION: Context & Objective**

**LEGAL POSITION:**
- The contract explicitly allows the Concessionaire to operate container trains on the Indian Railway network for both Export and Import Traffic as well as Domestic Traffic, subject to terms and conditions (Recital A).
- The contract mandates that the Concessionaire must abide by the terms and conditions laid down in the policy for movement of container trains on Indian Railways network (Recital D).
- The contract restricts the Concessionaire from commencing operations until entering into this Concession Agreement with the Railway Administration (Recital E).
- The contract explicitly states the objective to set out the mutual relationship and terms governing the right of the Concessionaire to offer its container trains for movement and the obligation of Railway Administration to

# Saving the analysis in word document

In [21]:
import json
from pathlib import Path
from docx import Document
from docx.shared import Pt


# =====================================================
# CONFIG
# =====================================================

DOCUMENT_NAME = name
JSON_DIR = Path("outputs") / DOCUMENT_NAME

OUTPUT_DOCX = Path(f"analysis_docs/{DOCUMENT_NAME}_PPP_Legal_Analysis.docx")


# =====================================================
# CREATE DOCUMENT
# =====================================================

doc = Document()

# Global font
style = doc.styles["Normal"]
style.font.name = "Calibri"
style.font.size = Pt(11)

# Title
title = doc.add_heading(
    f"PPP Concession Agreement Analysis\n{DOCUMENT_NAME}",
    level=0
)
title.alignment = 1  # center


# =====================================================
# HELPERS
# =====================================================

def add_heading(text, level=1):
    h = doc.add_heading(text, level=level)
    h.style.font.bold = True
    return h

def add_bullet(text):
    doc.add_paragraph(text, style="List Bullet")

def add_quote(text):
    p = doc.add_paragraph(text)
    p.paragraph_format.left_indent = Pt(24)
    p.paragraph_format.space_before = Pt(6)
    p.paragraph_format.space_after = Pt(6)
    p.runs[0].italic = True

def add_normal(text):
    doc.add_paragraph(text)


# =====================================================
# PROCESS ALL 12 SECTIONS
# =====================================================

json_files = sorted(JSON_DIR.glob("*.json"))

for json_file in json_files:
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    section_name = data["section_name"]
    analysis_md = data["analysis"]
    confidence = data.get("confidence", "UNKNOWN")

    # -----------------------------
    # Section Header
    # -----------------------------
    add_heading(section_name, level=1)

    # -----------------------------
    # Parse Markdown-like content
    # -----------------------------
    lines = analysis_md.split("\n")

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Skip duplicate SECTION line
        if line.startswith("SECTION:"):
            continue

        # Block headers
        if line.endswith(":") and line.isupper():
            add_heading(line.replace(":", "").title(), level=2)

        # Bullet points
        elif line.startswith("- "):
            add_bullet(line[2:].strip())

        # Clause quotes
        elif line.startswith("\"") or line.startswith("“"):
            add_quote(line)

        # Evidence strength (already inside analysis)
        elif line.startswith("EVIDENCE_STRENGTH"):
            add_heading("Evidence Strength", level=2)
            strength = line.split(":")[1].strip()
            p = add_normal(strength)
            p = doc.paragraphs[-1]
            p.runs[0].bold = True

        else:
            add_normal(line)

    # -----------------------------
    # Confidence Summary
    # -----------------------------
    add_heading("Confidence Assessment", level=2)
    p = add_normal(f"Overall Confidence Level: {confidence}")
    p = doc.paragraphs[-1]
    p.runs[0].bold = True

    # Page break after each section
    doc.add_page_break()


# =====================================================
# SAVE DOCUMENT
# =====================================================

doc.save(OUTPUT_DOCX)
print(f"✅ PPP Legal Memo generated: {OUTPUT_DOCX.resolve()}")


✅ PPP Legal Memo generated: /Users/happy/Desktop/iimmu/ppp_llm_work/analysis_docs/Adani  Mundra Port agreement for operations_PPP_Legal_Analysis.docx
